In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
learning_rate = 0.01
training_steps = 5000
batch_size = 128
display_step = 200

num_input = 28
time_steps = 28
num_hidden = 128
num_classes = 10

X = tf.placeholder(dtype=tf.float32, shape=[None, time_steps, num_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

weights = {
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
def BiRNN(x, weights, biases):
  x = tf.unstack(x, time_steps, 1)
  lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
  lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

  try:
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell,
                x, dtype=tf.float32)
  except Exception:
    outputs = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell,
                x, dtype=tf.float32)
  return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [4]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits=logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_op)

correct_pred = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will a

In [0]:
with tf.Session() as sess:
  sess.run(init)
  for step in range(1, training_steps+1):
    batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, num_input))
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})

    if step%display_step==0 or step==1:
      loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})

      print('Step '+str(step)+", Minibatch Loss = "+\
            "{:.4f}".format(loss)+", Training Accuracy = "+\
            "{:.3f}".format(acc))
  print("Optimization finished!")

  test_len = 128
  test_data = mnist.test.images[: test_len].reshape((-1, time_steps, num_input))
  test_label = mnist.test.labels[:test_len]
  test_acc = sess.run(accuracy, feed_dict={X: test_data, Y: test_label})
  print('Testing Accuracy:', test_acc)

Step 1, Minibatch Loss = 2.7708, Training Accuracy = 0.172
Step 200, Minibatch Loss = 1.2504, Training Accuracy = 0.633
Step 400, Minibatch Loss = 0.8814, Training Accuracy = 0.695
Step 600, Minibatch Loss = 0.8318, Training Accuracy = 0.781
Step 800, Minibatch Loss = 0.5620, Training Accuracy = 0.820
Step 1000, Minibatch Loss = 0.3655, Training Accuracy = 0.906
Step 1200, Minibatch Loss = 0.4065, Training Accuracy = 0.859
Step 1400, Minibatch Loss = 0.4376, Training Accuracy = 0.844
Step 1600, Minibatch Loss = 0.2584, Training Accuracy = 0.922
Step 1800, Minibatch Loss = 0.2058, Training Accuracy = 0.945
Step 2000, Minibatch Loss = 0.2226, Training Accuracy = 0.953
Step 2200, Minibatch Loss = 0.2567, Training Accuracy = 0.914
Step 2400, Minibatch Loss = 0.1983, Training Accuracy = 0.938
Step 2600, Minibatch Loss = 0.2214, Training Accuracy = 0.914
Step 2800, Minibatch Loss = 0.1573, Training Accuracy = 0.961
Step 3000, Minibatch Loss = 0.1914, Training Accuracy = 0.922
Step 3200, Mini